In [1]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.estimator import Estimator
from sagemaker.model import Model
from sagemaker.inputs import TrainingInput
from sagemaker.model_monitor import DataCaptureConfig

from sagemaker import get_execution_role

# role = "<sagemaker-execution-role>"

role=get_execution_role()
bucket = "feature-engineering-bucket-989220949c9c"
region = "ap-south-1"
image_uri_train = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-train:latest"

# Preprocessing Step
preprocess_processor = ScriptProcessor(
    image_uri=image_uri_train,
    role=role,
    command=["python3"],
    instance_count=1,
    instance_type="ml.t3.medium"
)

preprocess_step = ProcessingStep(
    name="preprocess_step",
    processor=preprocess_processor,
    code="preprocess.py",
    inputs=[
        ProcessingInput(
            source="s3://feature-engineering-bucket-989220949c9c/Dataset/bank-additional-full.csv",  # S3 input file
            destination="/opt/ml/processing/input/"  # Path inside the container
        )
        # {"source": "s3://feature-engineering-bucket/bank-additional-full.csv", "destination": "/opt/ml/processing/input"}
    ],
    outputs=[
        ProcessingOutput(
            output_name="processed_data",
            source="/opt/ml/processing/output",  # Path inside the container
            destination=f"s3://{bucket}/processed/"  # S3 output location
        )
    ]
)


# Define Estimator
estimator = Estimator(
    image_uri=image_uri_train,
    role=role,
    entry_point="train.py",             # ✅ Training script passed at runtime
    source_dir=".",                     # ✅ Folder where train.py is located
    instance_count=1,
    instance_type="ml.m5.large",
    base_job_name="mlops-train",
    output_path=f"s3://{bucket}/models/"  # ✅ where model.tar.gz will be saved
)

# Define TrainingStep in pipeline
training_step = TrainingStep(
    name="TrainingStep",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=f"s3://{bucket}/processed/bank-additional-processed.csv",
            content_type="text/csv"
        )
    }
)


# Register model
model_artifact = training_step.properties.ModelArtifacts.S3ModelArtifacts
model = Model(
    image_uri=image_uri_train,
    model_data=model_artifact,
    role=role
)

register_step = RegisterModel(
    name="RegisterModel",
    model=model,
    content_types=["text/csv"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name="mlops-model-group",
    approval_status="Approved"
)

# Define Pipeline
pipeline = Pipeline(
    name="MLOpsPipeline",
    steps=[preprocess_step, training_step, register_step]
)

# Deploy Pipeline
pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.wait()


# --------------------------------------------------------------------------------
# 5. Deploy the Model to Endpoint
# --------------------------------------------------------------------------------

image_uri_inference = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-inference:latest"

# Access the steps
steps = execution.list_steps()
s3_model_path = None

# Find the training step
for step in steps:
    if step["StepName"] == "TrainingStep":
        s3_model_path = step["Metadata"]["TrainingJob"]["ModelArtifacts"]["S3ModelArtifacts"]
        print("Model stored at:", s3_model_path)
        

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

model = Model(
    image_uri=image_uri_inference,
    model_data=s3_model_path,
    role=role
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint3",
    data_capture_config=DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=f"s3://{bucket}/monitoring/data/",
        capture_options=["Input", "Output"]
    )
)

response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print(response)



# --------------------------------------------------------------------------------
# 6. Monitor Performance Using Model Monitor
# --------------------------------------------------------------------------------


from sagemaker.model_monitor import DefaultModelMonitor


monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800
)


monitor.create_monitoring_schedule(
    monitor_schedule_name="mlops-monitor",
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=f"s3://{bucket}/monitoring/reports/",
    # generate the report every 5 minutes
    schedule_cron_expression="cron(0/5 * * * ? *)"
)

# Now performance reports will be generated daily in S3


# --------------------------------------------------------------------------------
# 7. Automate Retraining (Optional)
# --------------------------------------------------------------------------------


# Use EventBridge or S3 trigger → Lambda → start pipeline
# Lambda code (Python) can call:
# import boto3
# sm = boto3.client('sagemaker')
# sm.start_pipeline_execution(PipelineName='MLOpsPipeline')


sagemaker.config INFO - Fetched defaults config from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.Subnets
sag

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:100                                                                                  │
│                                                                                                  │
│    97 )                                                                                          │
│    98                                                                                            │
│    99 # Deploy Pipeline                                                                          │
│ ❱ 100 pipeline.upsert(role_arn=role)                                                             │
│   101 execution = pipeline.start()                                                               │
│   102 execution.wait()                                                                           │
│   103                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline.py:312 in upsert             │
│                                                                                                  │
│    309 │   │   │   # after fetching the config.                                                  │
│    310 │   │   │   raise ValueError("An AWS IAM role is required to create or update a Pipeline  │
│    311 │   │   try:                                                                              │
│ ❱  312 │   │   │   response = self.create(role_arn, description, tags, parallelism_config)       │
│    313 │   │   except ClientError as ce:                                                         │
│    314 │   │   │   error_code = ce.response["Error"]["Code"]                                     │
│    315 │   │   │   error_message = ce.response["Error"]["Message"]                               │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:171 in wrapper  │
│                                                                                                  │
│   168 │   │   │   │   │   caught_ex = e                                                          │
│   169 │   │   │   │   finally:                                                                   │
│   170 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 171 │   │   │   │   │   │   raise caught_ex                                                    │
│   172 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   173 │   │   │   else:                                                                          │
│   174 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:142 in wrapper  │
│                                                                                                  │
│   139 │   │   │   │   start_timer = perf_counter()                                               │
│   140 │   │   │   │   try:                                                                       │
│   141 │   │   │   │   │   # Call the original function                                           │
│ ❱ 142 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   143 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   144 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   145 │   │   │   │   │   extra += f"&x-latency={round(elap

In [14]:
from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

model = Model(
    image_uri=image_uri,
    model_data="s3://feature-engineering-bucket-989220949c9c/models/pipelines-80zrh3f0kyla-TrainingStep-ZYJh9bEUwc/output/model.tar.gz",  # use exact path
    role=role
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint4",
    data_capture_config=DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=f"s3://{bucket}/monitoring/data/",
        capture_options=["Input", "Output"]
    )
)


sagemaker.config INFO - Applied value from config key = SageMaker.Model.VpcConfig
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
------------------------------------------------*

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 │   role=role                                                                               │
│    8 )                                                                                           │
│    9                                                                                             │
│ ❱ 10 predictor = model.deploy(                                                                   │
│   11 │   initial_instance_count=1,                                                               │
│   12 │   instance_type="ml.m5.large",                                                            │
│   13 │   endpoint_name="mlops-prod-endpoint4",                                                   │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model.py:1814 in deploy                        │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/session.py:6250 in                             │
│ endpoint_from_production_variants                                                                │
│                                                                                                  │
│   6247 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6248 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6249 │   │                                                                                     │
│ ❱ 6250 │   │   return self.create_endpoint(                                                      │
│   6251 │   │   │   endpoint_name=name,                                                           │
│   6252 │   │   │   config_name=name,                                                             │
│   6253 │   │   │   tags=endpoint_tags,                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/session.py:5095 in create_endpoint             │
│                                                                                                  │
│   5092 │   │   │   logger.error(                                                                 │
│   5093 │   │   │   │   "Please check the troubleshooting guide for common errors: %s", troubles  │
│   5094 │   │   │   )                                                                             │
│ ❱ 5095 │   │   │   raise e                                                                       │
│   5096 │                                                                                         │
│   5097 │   def endpoint_in_service_or_not(self, endpoint_na

In [5]:
pipeline.list_steps()


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 pipeline.list_steps()                                                                        │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Pipeline' object has no attribute 'list_steps'

In [2]:
from sagemaker.model import Model
model = Model(
    image_uri=image_uri,
    model_data="s3://feature-engineering-bucket-989220949c9c/models/pipelines-80zrh3f0kyla-TrainingStep-ZYJh9bEUwc/model.tar.gz",
    role=role,
    entry_point="serve.py",      # ✅ send script at runtime
    source_dir="."               # ✅ folder containing serve.py
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-endpoint"
)


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.Model.VpcConfig
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   source_dir="."               # ✅ folder containing serve.py                            │
│   21 )                                                                                           │
│   22                                                                                             │
│ ❱ 23 predictor = model.deploy(                                                                   │
│   24 │   initial_instance_count=1,                                                               │
│   25 │   instance_type="ml.m5.large",                                                            │
│   26 │   endpoint_name="mlops-endpoint"                                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model.py:1737 in deploy                        │
│                                                                                                  │
│   1734 │   │   │   return None                                                                   │
│   1735 │   │                                                                                     │
│   1736 │   │   else:  # existing single model endpoint path                                      │
│ ❱ 1737 │   │   │   self._create_sagemaker_model(                                                 │
│   1738 │   │   │   │   instance_type=instance_type,                                              │
│   1739 │   │   │   │   accelerator_type=accelerator_type,                                        │
│   1740 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model.py:936 in _create_sagemaker_model        │
│                                                                                                  │
│    933 │   │   │   if self.name is None and model_package.name is not None:                      │
│    934 │   │   │   │   self.name = model_package.name                                            │
│    935 │   │   else:                                                                             │
│ ❱  936 │   │   │   container_def = self.prepare_container_def(                                   │
│    937 │   │   │   │   instance_type,                                                            │
│    938 │   │   │   │   accelerator_type=accelerator_type,                                        │
│    939 │   │   │   │   serverless_inference_config=serverless_inference_config,                  │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model.py:722 in prepare_container_def          │
│                                                                                                  │
│    719 │   │   )                                                                                 │
│    720 │   │   deploy_env = copy.deepcopy(self.env)                                              │
│    721 │   │   if self.source_dir or self.dependencies or self.entry_point or self.git_config:   │
│ ❱  722 │   │   │   self._upload_code(deploy_key_prefix, repack=self.is_repack())                 │
│    723 │   │   │   deploy_env.update(self._script_mode_env_vars())                               │
│    724 │   │                                                                                     │
│    725 │   │   return sagemaker.container_def(              